# Prac 08
In this prac we will be building a recommender system for movies.

# Q1
Using the [MovieLens](https://grouplens.org/datasets/movielens/) (small) data set complete the following:
- Download the [ml-latest-small](https://files.grouplens.org/datasets/movielens/ml-latest-small.zip) data file. Unzip, and then upload the files 'movies.csv' and 'ratings.csv' to the Google Colab
- Load the movies and ratings into a pandas data frame, and inspect the data
- Convert the ratings dataframe into a utility matrix using `df.pivot`, and inspect the matrix.
- Select a user from the utility matrix that we will recommend a movie for
- Predict the rating of a movie they have not seen using a user-based approach
- Predict the ratings of all unseen movies and recommend the top 10

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn

## Download data
Download the [ml-latest-small](https://files.grouplens.org/datasets/movielens/ml-latest-small.zip) data file. Unzip, and then upload the files 'movies.csv' and 'ratings.csv' to the Google Colab

Since the data is in a .zip file it's probably easiest to just use some bash tools to download and extract the files directly into the colab area.

In [ ]:
! [[ -e ml-latest-small.zip ]] || $(wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip; unzip ml-latest-small.zip)

## Load the data
Load the movies and ratings into a pandas data frame, and inspect the data.

In [ ]:
# load the movies data
movies_df = pd.read_csv('/content/ml-latest-small/movies.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# load the ratings data
ratings_df = pd.read_csv('/content/ml-latest-small/ratings.csv').drop(columns='timestamp')
ratings_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


## Create a utility matrix
Convert the ratings dataframe into a utility matrix using `df.pivot`, and inspect the matrix.
- Make the matrix smaller by selecting only movies with more than 50 ratings, and users who have rated more than 50 movies

In [ ]:
# convert ratings table into a utility matrix
utility = ratings_df.pivot(index='userId',
                           columns='movieId',
                           values='rating')
utility.tail()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
609,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
610,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
users_gt_50 = utility.count(axis='columns') > 50
movies_gt_50 = utility.count(axis='index') > 50

In [ ]:
utility_sm = utility.loc[users_gt_50, movies_gt_50]
utility_sm

movieId,1,2,3,6,7,10,11,16,17,19,...,81845,89745,91500,91529,99114,106782,109374,109487,112852,122904
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,4.0,5.0,4.0,4.0,3.0,4.0,4.0,4.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,5.0,NaN,1.0,0.5,0.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,4.0,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
606,2.5,NaN,NaN,NaN,2.5,NaN,2.5,NaN,4.0,2.0,...,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Select a user
Select a user from the utility matrix that we will recommend a movie for.

I recommend chosing user 605 in the above list.

In [ ]:
User1 = utility.loc[605]

## Predict the rating of a movie
Predict the rating of a movie they have not seen using a user-based approach
- use a similarity score of `0.8` to select similar users
- predict the rating for movie with `id=7`

In [ ]:
# using row1.corr(row2) we calculate the pearson correlation function between users
def corrUser(row):
  if User1.equals(row): # For later implementation we don't want to compare a user to themselves.
    return 0            # Instead of skipping them we just set the correlation to zero.
  return User1.corr(row)

In [ ]:
# now compute all the similarities
similarities = utility.apply(corrUser,
                             axis=1)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [ ]:
threshold=0.8
peers_id = utility.index[similarities>threshold]
print(f"There are {len(peers_id)} similar users")

There are 15 similar users


In [ ]:
utility.loc[peers_id]

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
389,5.0,NaN,NaN,NaN,4.0,5.0,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
396,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Predicted score is
$\frac{\sum( S_i * x_i) }{ \sum (S_i)}$
where $S_i$ is the similarity score and $x_i$ is the user rating.

In [ ]:
# Compute the average of similar user scores for movie with id=1
mid = 1
numerator = np.sum(similarities[peers_id] * utility.loc[peers_id, mid])
denomenator = np.sum( similarities[peers_id] * (utility.loc[peers_id, mid] > 0) )
predicted_score = numerator / denomenator
predicted_score

4.650483330913248

## Select top 10 predictions
Predict the ratings of all unseen movies and recommend the top 10
- Use the above to write a function to predict the score for all the movies
- sort the predictions and choose the top 10
- report the predicted score and names of the top 10 movies

In [ ]:
def predict_scores(peers, utility):
  """
  Predict the score for all unseen movies.
  Peers should list the userid for all the similar users.
  Utility should be a utility matrix.

  Returns a dict of {movieId:score}
  """
  scores = {}
  for movie in utility.columns:
    if np.isnan(User1[movie]):     # only predict scores for unseen movies
      numerator = np.sum(similarities[peers_id] * utility.loc[peers_id, movie])
      denomenator = np.sum( similarities[peers_id] * (utility.loc[peers_id, movie] > 0) )
      predicted_score = numerator / denomenator
      if not np.isnan(predicted_score):  # don't record predictions of nan
        scores[movie] = predicted_score
  return scores

In [ ]:
# compute the predicted scores
predictions = predict_scores(peers_id,
                             utility)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


In [ ]:
# choose the top10 movieIds
top_10 = sorted(predictions,
                key=predictions.get, # sort based on the value not the key
                reverse=True, # reverse means hightest first
                )[:10]  # choose first 10
top_10

[6, 62, 140, 141, 216, 293, 520, 628, 736, 750]

In [ ]:
# Display a nice table of results
print("Pred \t| Movie")
print("--------------")
for movie in top_10:
  # extract the movie name from the movies_df
  name = movies_df['title'][movies_df['movieId']==movie].values[0]
  print(f"{predictions[movie]:3.1f}\t| {name}")

Pred 	| Movie
--------------
5.0	| Heat (1995)
5.0	| Mr. Holland's Opus (1995)
5.0	| Up Close and Personal (1996)
5.0	| Birdcage, The (1996)
5.0	| Billy Madison (1995)
5.0	| Léon: The Professional (a.k.a. The Professional) (Léon) (1994)
5.0	| Robin Hood: Men in Tights (1993)
5.0	| Primal Fear (1996)
5.0	| Twister (1996)
5.0	| Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)
